In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
mistral_api_key = os.environ["MISTRAL_API_KEY"]

In [331]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    mistral_api_key=mistral_api_key,
   # model="mistral-large-latest",
    model="pixtral-12b-2409",
    temperature=0,
    max_retries=2,
    max_tokens = 1024
    # other params...
)

In [333]:
from PIL import Image
def reduceImageSize(imageFile):
    file_path = "./data/temp.jpg"
    img = Image.open(imageFile)
    img.thumbnail((512, 512))  # Adjust the size as needed
    img.save(file_path)
    return file_path

In [349]:
import base64
import os

def encode_image(image_path):
    """Getting the base64 string"""
    file_path = reduceImageSize(image_path)
    with open(file_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
        #return base64.b64encode(img).decode("utf-8")


In [352]:
from langchain_core.messages import HumanMessage

def image_summarize(img_base64, prompt):
    """Make image summary"""

    msg = llm.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content

In [356]:
from langchain_core.prompts import ChatPromptTemplate
template = """You are an expert to read the image and summerize it. While summarization, read all details including texts in it.
Give a concise summary of the image and also explain the summary of it. Summary of image should give the details of purpose of image 
as well as about the image. Reply the query if there is any
{query}
"""

In [347]:
base64_image = encode_image("./data/amazon.jpeg")
query = "Let me know the price of each items in Indian rupees in tabular format"
prompt = template.format(query=query)
result = image_summarize(base64_image, prompt)
result

'The image is a screenshot of a grocery delivery app interface. It displays a variety of fruits and vegetables available for purchase, along with their prices in Indian Rupees (INR). The interface includes categories such as "All vegetables," "All fruits," and "Leafy greens." There is also an option to search for specific items and a section for freshly cut and sprouted items. The app offers delivery options and shows the delivery fee.\n\nHere is the price list of the items in the image in Indian Rupees (INR):\n\n| Item Name                      | Price (INR) |\n|--------------------------------|-------------|\n| Fresh Apple Red Delicious      | 197         |\n| Fresh Dragon Fruit White       | 50          |\n| Fresh Apple Red Delicious      | 97          |\n| Freshly Cut Cauliflower        | 117         |\n| Freshly Cut Carrots            | 45          |\n| Freshly Cut Beetroot           | 50          |\n\nThese prices are for specific quantities and qualities of the items as mentione

In [360]:
base64_image = encode_image("./data/figure-2-1.jpg")
query = ""
prompt = template.format(query=query)
result = image_summarize(base64_image, prompt)
result

'The image depicts a flowchart outlining a process for improving a language model using Reinforcement Learning from Human Feedback (RLHF). Here is a detailed summary of the image:\n\n### Summary:\nThe flowchart illustrates a cyclical process aimed at refining a language model through iterative feedback and training. The process begins with an initial language model, referred to as the "Initial RLHF Model." This model generates responses to prompts, which are then evaluated by human feedback to determine their helpfulness, harmlessness, and other criteria. This feedback is used to fine-tune the model, creating a "Finetuned Initial Model."\n\nThe finetuned model is then subjected to further evaluation and training. Responses generated by the finetuned model are compared to those from a "Finetuned Performance Model," which is presumably a more advanced version of the initial model. This comparison involves a process called "Constitutional AI Feedback for Self-Improvement," where the model